# CONSUMER

#### 3.8 Consumo de datos desde Kafka

En esta etapa se realiza el consumo de datos en tiempo real utilizando **Apache Kafka**. Para ello, se importan las librerías necesarias, incluyendo `KafkaConsumer` y PySpark. Posteriormente, se configura el consumidor estableciendo la conexión con el servidor de Kafka y se define el proceso de deserialización de los mensajes, permitiendo transformar los datos recibidos en un formato adecuado para su posterior procesamiento y análisis.

In [ ]:
from kafka import KafkaConsumer
import json
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import clear_output, display

# =========================
# Constantes de conexión
# =========================
CASSANDRA_HOST = "100.68.89.127"
CASSANDRA_PORT = "9042"

KAFKA_HOST = "100.68.89.127"
KAFKA_PORT = "9092"
KAFKA_TOPIC = "uber_trips"
KAFKA_GROUP_ID = "grupo_analisis"

# =========================
# Configuración del Consumer
# =========================
consumer = KafkaConsumer(
    KAFKA_TOPIC,
    bootstrap_servers=f"{KAFKA_HOST}:{KAFKA_PORT}",
    auto_offset_reset="earliest",
    group_id=KAFKA_GROUP_ID,
    value_deserializer=lambda m: json.loads(m.decode("utf-8")),
    enable_auto_commit=True
)

print("Kafka Consumer inicializado correctamente")
print(consumer)


In [ ]:
from pyspark.sql import SparkSession

# =========================
# Crear SparkSession
# =========================
spark = SparkSession.builder \
    .appName("KafkaTripsConsumer") \
    .getOrCreate()

# =========================
# Cargar archivo de zonas
# =========================
loc_raw = spark.read.csv(
    "data/taxi_zone_lookup.csv",
    header=True,
    inferSchema=True
)

# =========================
# Limpieza de columnas
# =========================
loc = loc_raw.drop("Borough", "service_zone")

# =========================
# Preparación de DataFrames para joins
# =========================
loc_pickup = loc.withColumnRenamed("Zone", "pickup_zone")
loc_delivery = loc.withColumnRenamed("Zone", "delivery_zone")

# =========================
# Verificación
# =========================
loc_pickup.show(5)
loc_pickup.printSchema()


#### 3.9 Almacenamiento y procesamiento de datos

En esta etapa se realiza el consumo de datos provenientes de **Apache Kafka** mediante el uso de `KafkaConsumer`. A partir de los mensajes recibidos, se construye un DataFrame que permite almacenar los datos y aplicar las transformaciones requeridas para el análisis.

Los datos provenientes de Kafka se leen dentro de un bucle de consumo continuo y se almacenan temporalmente en una estructura denominada `data_batch`. Con el fin de optimizar el procesamiento, se limita el número de registros por lote, y mediante una lógica condicional se transforma cada lote de datos en un DataFrame de Spark sobre el cual se aplican las siguientes operaciones:

- Limpieza de registros con valores nulos.
- Unión (*join*) con la tabla de ubicaciones utilizando los identificadores de origen y destino de los viajes.
- Cálculo de las ganancias de la plataforma por cada viaje.
- Cálculo del acumulado de las ganancias generadas por la plataforma.


In [ ]:
from pyspark.sql.types import (
    StructType, StructField,
    IntegerType, StringType, FloatType
)
from pyspark.sql.functions import col
from datetime import datetime

# =========================
# Esquema de los datos Kafka
# =========================
schema = StructType([
    StructField("index_trip", IntegerType(), True),
    StructField("request_datetime", StringType(), True),
    StructField("pickup_datetime", StringType(), True),
    StructField("dropoff_datetime", StringType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("base_passenger_fare", FloatType(), True),
    StructField("tips", FloatType(), True),
    StructField("driver_pay", FloatType(), True),
    StructField("hour", FloatType(), True),
    StructField("year", FloatType(), True),
    StructField("month", FloatType(), True),
    StructField("day", FloatType(), True),
    StructField("on_time_pickup", IntegerType(), True)
])

# =========================
# Configuración de batch
# =========================
data_batch = []
BATCH_SIZE = 1   # Ajustable según volumen y recursos

# =========================
# Consumo de mensajes Kafka
# =========================
for message in consumer:
    record = message.value
    data_batch.append(record)

    print(record)

    # =========================
    # Procesar lote
    # =========================
    if len(data_batch) >= BATCH_SIZE:
        print("✔ Lote completo. Procesando datos...")

        # Crear DataFrame Spark
        df = spark.createDataFrame(data_batch, schema)

        # Limpieza de registros nulos
        df_clean = df.dropna()

        # Cálculo de ganancias de la plataforma
        df_sales = df_clean.withColumn(
            "uber_sales",
            col("base_passenger_fare") + col("tips") - col("driver_pay")
        )

        # Join con tabla de zonas de inicio
        df_pickup = df_sales.join(
            loc_1,
            df_sales.PULocationID == loc_1.LocationID,
            how="left"
        ).drop("LocationID")

        # Join con tabla de zonas de destino
        df_final = df_pickup.join(
            loc_2,
            df_pickup.DOLocationID == loc_2.LocationID,
            how="left"
        ).drop("LocationID")

        # Agregación de ganancias por fecha y hora
        df_agg = df_final.groupBy(
            "year", "month", "day", "hour"
        ).sum("uber_sales")

        # =========================
        # Persistencia de resultados
        # =========================
        df_final.write \
            .mode("append") \
            .option("header", True) \
            .csv("Proyecto_Final/clean_data.csv")

        # df_agg.write \
        #     .mode("append") \
        #     .option("header", True) \
        #     .csv("Proyecto_Final/aggregated_data.csv")

        # Limpiar batch
        data_batch.clear()


In [ ]:
df_merged4.show()

In [ ]:
df_merged4.explain()